## Processing ETL des sets de données Olist

#### 1. Nettoyage: Opérations globales à tous les fichiers source

Dans cette partie je gère les nettoyages qui peuvent potentiellement s'appliquer à toutes les tables, sans faire de traitement spécifique pour telle ou telle table

In [22]:
# Import de dépendances et chargement des datasets en mémoire vive

import os
from pathlib import Path
import pandas as pd
from scripts.utils import clean_data

data = {}

rt = Path('./data')
file_paths = os.listdir(rt)
csv_file_paths = [f for f in file_paths if f.endswith('.csv')]
for csv_path in csv_file_paths:
    print(f'Loading {csv_path}...')
    df = pd.read_csv(rt.joinpath(csv_path))
    df = clean_data(df) # Utilisation d'un heler générique de nettoyage
    source = csv_path.replace('olist_', '').replace('.csv', '').replace('_dataset', '')
    data[source] = df

Loading olist_orders_dataset.csv...


/home/mathieu/prj/olist-learning/scripts/utils.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col],
/home/mathieu/prj/olist-learning/scripts/utils.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col],
/home/mathieu/prj/olist-learning/scripts/utils.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely

Loading olist_products_dataset.csv...
Loading olist_order_items_dataset.csv...
Loading product_category_name_translation.csv...
Loading olist_sellers_dataset.csv...
Loading olist_geolocation_dataset.csv...
Loading olist_order_reviews_dataset.csv...
Loading olist_order_payments_dataset.csv...
Loading olist_customers_dataset.csv...


/home/mathieu/prj/olist-learning/scripts/utils.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col],
/home/mathieu/prj/olist-learning/scripts/utils.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col],


#### 2. Nettoyage: Cas spécifiques

Dans cette partie du notebook j'applique des netoyages plus spécifiques, sur des tables en particulier:
- customers: suppression de doublons et d"une colonne non utilisée => EDIT: Pas une bonne idée, je commente
- geolocation et customers: normalisation des noms de ville pour cohérence des éventuels PKI qui utiliseraient les noms de ville

In [ ]:
# Nettoyage des doublons de données table customers
# data['customers'].drop_duplicates(subset=['customer_unique_id'], keep='first')
# data['customers'] = data['customers'].drop(columns=['customer_unique_id'])

In [23]:
from scripts.utils import norm

# Normalise les noms de villes
data['geolocation']['geolocation_city'] = data['geolocation']['geolocation_city'].apply(norm)
data['customers']['customer_city'] = data['customers']['customer_city'].apply(norm)

#### 3. Transformations

La transformation principale que j'ai imaginée sur ce cas d'école a été d'ajouter les traductions de noms de catégories directement dans la table produits.

Le but étant de simplifier les requêtes devant filtrer ou grouper par catégorie, cela évite d'avoir à faire des jointures ultérieures pour récupérer les traductions.

Au départ j'ai même considéré faire cela pour cmplètement supprimer la table de traductions, mais j'ai décidé de la garder, dans le scénario où un utilisateur brésilien aurait besoin de faire des requêtes sur la base de données: dans ce cas les jointures resteront faisables.

In [24]:
# Transformation de la table des produits

# Merge des dataframes pour ajouter la traduction anglaise et éviter les jointures
data['products'] = data['products'].merge(
    data['product_category_name_translation'],
    how='left', # Type de jointure : left pour conserver tous les produits même sans traduction
    left_on='product_category_name', # Clé de jointure dans la table des produits
    right_on='product_category_name' # Clé de jointure dans la table de traduction
)

# Renommer la nouvelle colonne pour éviter les conflits
data['products'].rename(columns={'product_category_name_english': 'product_category_name_en'},
                inplace=True)

# Résultat final
print(data['products'][['product_category_name', 'product_category_name_en']].head())

   product_category_name product_category_name_en
0             perfumaria                perfumery
1                  artes                      art
2          esporte_lazer           sports_leisure
3                  bebes                     baby
4  utilidades_domesticas               housewares


#### 4. Chargement en base de données


Pour le chargement j'ai choisi d'utiliser sqlite3 pour gérer la partie création de tables en manuel avec des scripts SQL; ça permet d'avoir une création de tables qui inclue directement toutes les contraintes nécessaires, et c'est plus simple que d'utiliser la syntaxe de sqlalchemy, qui ajoute une couche d'abstraction supplémentaire à SQL.

In [25]:
import sqlite3

DB_PATH = "./olist.db"
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()

    # Ouverture et exécution du script SQL de création de schéma de base de données
    with open('./scripts/schema.sql', 'r') as f:
        schema_sql = f.read()
        cur.executescript(schema_sql)

In [26]:
# Chargement des données nettoyées/transformées dans la base de données SQLite
for table_name, df in data.items():
    df.to_sql(table_name, con=conn, if_exists='replace', index=False)

### 5. Développement d'indicateurs de perfomances

In [27]:
# Setup du système de benchmark
%load_ext autoreload
%autoreload 2

from scripts.utils import QueryBenchmark
import importlib
import scripts
importlib.reload(scripts.utils) # Recharger le module pour prendre en compte les modifications récentes
benchmark = QueryBenchmark()
format_stats = lambda rows, elapsed: f"Fetched {rows} rows in {elapsed:.3f}s"

from scripts.utils import get_perf_stats

# Définition d'une fonction d'exécution de requêtes SQL avec mesure de performance
def execute(fld, category, query, optimized=False, preview=False):
    query_type = "optimized" if optimized else "raw"
    with open(f'./scripts/{fld}/{category}/{query}.sql', 'r') as f:
        sql_script = f.read()
        result, rows_processed, execution_time = get_perf_stats(sql_script)
        benchmark.set_stats(category, query, query_type, rows_processed, execution_time)
        
        print(format_stats(rows_processed, execution_time))
        if preview:
            print('Résultat (max. 12):')
            for row in result:
                print(dict(row))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### 💰 Ventes

In [28]:
execute('pki', 'sales', 'daily', preview=True)

Fetched 633 rows in 0.420s
Résultat (max. 12):
{'jour': '2016-09-04', 'ca_jour': 136.23}
{'jour': '2016-09-05', 'ca_jour': 75.06}
{'jour': '2016-09-13', 'ca_jour': 40.95}
{'jour': '2016-10-02', 'ca_jour': 109.34}
{'jour': '2016-10-03', 'ca_jour': 595.14}
{'jour': '2016-10-04', 'ca_jour': 11914.65}
{'jour': '2016-10-05', 'ca_jour': 10212.22}
{'jour': '2016-10-06', 'ca_jour': 9398.61}
{'jour': '2016-10-07', 'ca_jour': 8255.19}
{'jour': '2016-10-08', 'ca_jour': 9716.13}
{'jour': '2016-10-09', 'ca_jour': 4152.11}
{'jour': '2016-10-10', 'ca_jour': 4675.1}


In [29]:

execute('pki', 'sales', 'monthly', preview=True)

Fetched 25 rows in 0.436s
Résultat (max. 12):
{'mois': '2016-09', 'ca_mois': 252.24}
{'mois': '2016-10', 'ca_mois': 59090.48}
{'mois': '2016-12', 'ca_mois': 19.62}
{'mois': '2017-01', 'ca_mois': 138488.04}
{'mois': '2017-02', 'ca_mois': 291908.01}
{'mois': '2017-03', 'ca_mois': 449863.6}
{'mois': '2017-04', 'ca_mois': 417788.03}
{'mois': '2017-05', 'ca_mois': 592918.82}
{'mois': '2017-06', 'ca_mois': 511276.38}
{'mois': '2017-07', 'ca_mois': 592382.92}
{'mois': '2017-08', 'ca_mois': 674396.32}
{'mois': '2017-09', 'ca_mois': 727762.45}


In [30]:
execute('pki', 'sales', 'yearly', preview=True)

Fetched 3 rows in 0.400s
Résultat (max. 12):
{'annee': '2016', 'ca_annee': 59362.34}
{'annee': '2017', 'ca_annee': 7249746.73}
{'annee': '2018', 'ca_annee': 8699763.05}


In [31]:
execute('pki', 'sales', 'previous_year_comparison', preview=True)

Fetched 633 rows in 0.811s
Résultat (max. 12):
{'jour': '2016-09-04', 'ca_courant': 136.23, 'ca_n_1': 28719.24, 'variation_abs': -28583.010000000002, 'variation_pct': -99.53}
{'jour': '2016-09-05', 'ca_courant': 75.06, 'ca_n_1': 24954.68, 'variation_abs': -24879.62, 'variation_pct': -99.7}
{'jour': '2016-09-13', 'ca_courant': 40.95, 'ca_n_1': 32054.37, 'variation_abs': -32013.42, 'variation_pct': -99.87}
{'jour': '2016-10-02', 'ca_courant': 109.34, 'ca_n_1': 25565.11, 'variation_abs': -25455.77, 'variation_pct': -99.57}
{'jour': '2016-10-03', 'ca_courant': 595.14, 'ca_n_1': 30389.12, 'variation_abs': -29793.98, 'variation_pct': -98.04}
{'jour': '2016-10-04', 'ca_courant': 11914.65, 'ca_n_1': 24787.58, 'variation_abs': -12872.930000000002, 'variation_pct': -51.93}
{'jour': '2016-10-05', 'ca_courant': 10212.22, 'ca_n_1': 29129.83, 'variation_abs': -18917.61, 'variation_pct': -64.94}
{'jour': '2016-10-06', 'ca_courant': 9398.61, 'ca_n_1': 21890.58, 'variation_abs': -12491.970000000001, 'v

In [32]:
execute('pki', 'sales', 'top10', preview=True)

Fetched 10 rows in 0.671s
Résultat (max. 12):
{'product_id': '5769ef0a239114ac3a854af00df129e4', 'product_category_name_en': 'fixed_telephony', 'product_name_length': 39.0, 'ca_produit': 109312.64, 'rang': 1}
{'product_id': 'bb50f2e236e5eea0100680137654686c', 'product_category_name_en': 'health_beauty', 'product_name_length': 60.0, 'ca_produit': 81887.42, 'rang': 2}
{'product_id': '422879e10f46682990de24d770e7f83d', 'product_category_name_en': 'garden_tools', 'product_name_length': 56.0, 'ca_produit': 79512.22, 'rang': 3}
{'product_id': 'd1c427060a0f73f6b889a5c7c61f2ac4', 'product_category_name_en': 'computers_accessories', 'product_name_length': 59.0, 'ca_produit': 70557.9, 'rang': 4}
{'product_id': '6cdd53843498f92890544667809f1595', 'product_category_name_en': 'health_beauty', 'product_name_length': 51.0, 'ca_produit': 64825.67, 'rang': 5}
{'product_id': 'd5991653e037ccb7af6ed7d94246b249', 'product_category_name_en': 'computers_accessories', 'product_name_length': 42.0, 'ca_produit'

#### 👥 Clients

In [33]:
execute('pki', 'customers', 'new_vs_recurring_customers', preview=True)

Fetched 1 rows in 0.062s
Résultat (max. 12):
{'new_customers': 99441, 'returning_customers': 0}


In [34]:
execute('pki', 'customers', 'average_cart', preview=True)

Fetched 1 rows in 0.053s
Résultat (max. 12):
{'avg_basket_value': 160.57763809214927}


In [35]:
execute('pki', 'customers', 'conversion_rate', preview=True)

Fetched 1 rows in 0.013s
Résultat (max. 12):
{'conversion_rate': 0.9702034372140264}


In [36]:
execute('pki', 'customers', 'rfmbbq_analysis', preview=True)

Fetched 98666 rows in 0.517s
Résultat (max. 12):
{'customer_id': '1617b1357756262bfa56ab541c47bc16', 'recency_days': 3059, 'frequency': 1, 'monetary': 13664.08}
{'customer_id': 'ec5b2ba62e574342386871631fafd3fc', 'recency_days': 2770, 'frequency': 1, 'monetary': 7274.88}
{'customer_id': 'c6e2731c5b391845f6800c97401a43a9', 'recency_days': 3288, 'frequency': 1, 'monetary': 6929.31}
{'customer_id': 'f48d464a0baaea338cb25f816991ab1f', 'recency_days': 2760, 'frequency': 1, 'monetary': 6922.21}
{'customer_id': '3fd6777bbce08a352fddd04e4a7cc8f6', 'recency_days': 3187, 'frequency': 1, 'monetary': 6726.66}
{'customer_id': '05455dfa7cd02f13d132aa7a6a9729c6', 'recency_days': 3003, 'frequency': 1, 'monetary': 6081.54}
{'customer_id': 'df55c14d1476a9a3467f131269c2477f', 'recency_days': 3240, 'frequency': 1, 'monetary': 4950.34}
{'customer_id': 'e0a2412720e9ea4f26c1ac985f6a7358', 'recency_days': 2773, 'frequency': 1, 'monetary': 4809.44}
{'customer_id': '24bbf5fd2f2e1b359ee7de94defc4a15', 'recency_d

### 📊 Cohortes

In [37]:
execute('pki', 'cohorts', 'first_month_retention', preview=True)

Fetched 634 rows in 0.422s
Résultat (max. 12):
{'cohort_date': '2016-09-04', 'cohort_month': 0, 'active_customers': 1}
{'cohort_date': '2016-09-05', 'cohort_month': 0, 'active_customers': 1}
{'cohort_date': '2016-09-13', 'cohort_month': 0, 'active_customers': 1}
{'cohort_date': '2016-09-15', 'cohort_month': 0, 'active_customers': 1}
{'cohort_date': '2016-10-02', 'cohort_month': 0, 'active_customers': 1}
{'cohort_date': '2016-10-03', 'cohort_month': 0, 'active_customers': 8}
{'cohort_date': '2016-10-04', 'cohort_month': 0, 'active_customers': 63}
{'cohort_date': '2016-10-05', 'cohort_month': 0, 'active_customers': 47}
{'cohort_date': '2016-10-06', 'cohort_month': 0, 'active_customers': 51}
{'cohort_date': '2016-10-07', 'cohort_month': 0, 'active_customers': 46}
{'cohort_date': '2016-10-08', 'cohort_month': 0, 'active_customers': 42}
{'cohort_date': '2016-10-09', 'cohort_month': 0, 'active_customers': 26}


In [38]:
execute('pki', 'cohorts', 'lifetime_value_per_cohort', preview=True)

Fetched 616 rows in 0.759s
Résultat (max. 12):
{'cohort_date': '2016-09-04', 'customers_in_cohort': 1, 'total_revenue': 136.23, 'ltv_per_customer': 136.23}
{'cohort_date': '2016-09-05', 'customers_in_cohort': 1, 'total_revenue': 75.06, 'ltv_per_customer': 75.06}
{'cohort_date': '2016-09-15', 'customers_in_cohort': 1, 'total_revenue': 143.46, 'ltv_per_customer': 143.46}
{'cohort_date': '2016-10-02', 'customers_in_cohort': 1, 'total_revenue': 109.34, 'ltv_per_customer': 109.34}
{'cohort_date': '2016-10-03', 'customers_in_cohort': 8, 'total_revenue': 595.14, 'ltv_per_customer': 74.3925}
{'cohort_date': '2016-10-04', 'customers_in_cohort': 60, 'total_revenue': 11295.48, 'ltv_per_customer': 188.25799999999998}
{'cohort_date': '2016-10-05', 'customers_in_cohort': 42, 'total_revenue': 9645.94, 'ltv_per_customer': 229.6652380952381}
{'cohort_date': '2016-10-06', 'customers_in_cohort': 49, 'total_revenue': 9131.23, 'ltv_per_customer': 186.35163265306122}
{'cohort_date': '2016-10-07', 'customers

### 6. Optimisation base & requêtes

In [39]:
import sqlite3

# Optimisation de la base
# Ajout des indexes pour optimiser les performances des requêtes
with open('./scripts/indexes.sql', 'r') as f:
    sql_script = f.read()
    with sqlite3.connect('./olist.db') as conn:
        cur = conn.cursor()
        cur.executescript(sql_script)   

### 7. Requêtes optimisées 🚀

#### 🚀 💰 Ventes

1 - CA journalier

Problèmes potentiels
- DATE() casse l’usage d’index
- Ordre des opérations:
    SQLite:
    - Fait le JOIN
    - Calcule DATE()
    - Groupe
    - Trie

Si les tables sont grosses, ça devient coûteux.

Optimisations:
- Éviter le calcul répétitif dans GROUP BY

SQLite va recalculer STRFTIME('%Y-%m', o.order_purchase_timestamp) pour chaque ligne du GROUP BY. On peut utiliser une sous-requête ou CTE pour calculer le mois une seule fois.

In [40]:
category = "sales"
query = "daily"

execute('opti_pki', category, query, optimized=True)

comparison = benchmark.compare(category, query)
print(comparison)

Fetched 633 rows in 0.388s
{'rows_difference': 0, 'time_difference': 0.03152861399939866, 'rows_improvement_percent': 0.0, 'time_improvement_percent': 7.5126608194956175}


2 - CA mensuel

Si order_payments a plusieurs paiements par commande, on peut pré-agréger les paiements avant le JOIN, ce qui réduit le nombre de lignes à grouper

Avantage: si order_payments a beaucoup de lignes par commande, ça réduit drastiquement le nombre de lignes manipulées par le GROUP BY.

In [41]:
category = "sales"
query = "monthly"

execute('opti_pki', category, query, optimized=True)

comparison = benchmark.compare(category, query)
print(comparison)

Fetched 25 rows in 0.428s
{'rows_difference': 0, 'time_difference': 0.008011595000425586, 'rows_improvement_percent': 0.0, 'time_improvement_percent': 1.838586052097372}


3 - CA annuel

Optimisation n°1 : Pré-agréger les paiements
Si une commande peut avoir plusieurs paiements, un JOIN multiplie les lignes avant agrégation.

Optimisation n°2 : Éviter DATE() si possible
- Si le timestamp est au format ISO (YYYY-MM-DD HH:MM:SS), on peut faire : SUBSTR(o.order_purchase_timestamp, 1, 10)
- SUBSTR(...,1,4) remplace STRFTIME('%Y', ...) et est plus rapide sur SQLite.
- WITH payments réduit le volume intermédiaire si commandes ont plusieurs paiements.

In [42]:
category = "sales"
query = "yearly"

execute('opti_pki', category, query, optimized=True)

comparison = benchmark.compare(category, query)
print(comparison)

Fetched 3 rows in 0.380s
{'rows_difference': 0, 'time_difference': 0.019855563004966825, 'rows_improvement_percent': 0.0, 'time_improvement_percent': 4.96436489768988}


4 - Chiffre d'affaires annuel comparé à n-1

Améliorations:
- Pré-agrégation des paiements par commande et en utilisant SUBSTR pour éviter DATE().
- Utilisation d'une CTE en lieu et place d'une sous-requête

Avantages:
- Volume intermédiaire réduit → on agrège order_payments par order_id avant le JOIN.
- SUBSTR(...,1,10) permet d’utiliser un index sur la colonne order_purchase_timestamp.
- Même logique que ta version originale, mais beaucoup plus performante sur de grandes tables.

In [43]:
category = "sales"
query = "previous_year_comparison"

execute('opti_pki', category, query, optimized=True)

comparison = benchmark.compare(category, query)
print(comparison)

Fetched 633 rows in 0.399s
{'rows_difference': 0, 'time_difference': 0.4110897630052932, 'rows_improvement_percent': 0.0, 'time_improvement_percent': 50.71922728683089}


5 - Top 10 produits par chiffre d’affaires

Problèmes de la version initiale :
- Jointure multiplicative entre order_items et order_payments → explosion du nombre de lignes intermédiaires (produits × paiements).
- Agrégation tardive (GROUP BY après tous les JOIN) → gros volume à trier et regrouper.
- ROW_NUMBER() + ORDER BY → double tri coûteux.
- Mauvaise scalabilité sur gros volumes.

Améliorations :
- Pré-agrégation des paiements par order_id avant les JOIN.
- Réduction du dataset le plus tôt possible (principe de “reduce early”).
- Application des classements et tris sur un dataset déjà agrégé.

Avantages :

- Volume intermédiaire fortement réduit.
- Suppression de l’effet multiplicatif paiements × produits.
- Moins de tris mémoire.
- Requête plus stable et scalable sur grandes tables.

In [44]:
category = "sales"
query = "top10"

execute('opti_pki', category, query, optimized=True)

comparison = benchmark.compare(category, query)
print(comparison)

Fetched 633 rows in 0.400s
{'rows_difference': -623, 'time_difference': 0.27159139800278354, 'rows_improvement_percent': -6230.0, 'time_improvement_percent': 40.463049731712644}


#### 🚀 👥 Clients

1 - Comptage de clients nouveaux vs récurrents

Améliorations:
- SUBSTR(...,1,10) remplace date() → plus rapide et indexable.
- Même logique de groupements, mais simplifiée pour éviter des fonctions SQLite lourdes.
- COUNT(DISTINCT customer_id) pour éviter double comptage si un client a plusieurs commandes dans la cohorte.

In [45]:
category = "customers"
query = "new_vs_recurring_customers"

execute('opti_pki', category, query, optimized=True)

comparison = benchmark.compare(category, query)
print(comparison)

Fetched 1 rows in 0.020s
{'rows_difference': 0, 'time_difference': 0.04239232899635681, 'rows_improvement_percent': 0.0, 'time_improvement_percent': 67.89987517320534}


2 - Montant de panier moyen

Point à optimiser:
En faisant un CTE (WITH) mais pour une seule utilisation, SQLite le traite parfois comme une sous-requête matérialisée, donc sur très gros volumes ça peut être lent.
Utiliser COUNT(CASE...) en lieu et place de SUM(CASE...)

Pourquoi c’est plus rapide:
- Pas de CTE, donc SQLite peut optimiser la sous-requête directement.
- GROUP BY order_id est indexable.

In [46]:
category = "customers"
query = "average_cart"

execute('opti_pki', category, query, optimized=True)

comparison = benchmark.compare(category, query)
print(comparison)

Fetched 1 rows in 0.054s
{'rows_difference': 0, 'time_difference': -0.001058970003214199, 'rows_improvement_percent': 0.0, 'time_improvement_percent': -1.9983827232773854}


3 - Taux de conversion

- COUNT(CASE WHEN ... THEN 1 END) compte uniquement les commandes livrées.
- * 1.0 force le calcul en float pour ne pas avoir un résultat entier (important en SQLite).
- ROUND(...,4) permet de limiter le nombre de décimales à 4 (optionnel mais pratique pour affichage).

In [47]:
category = "customers"
query = "conversion_rate"

execute('opti_pki', category, query, optimized=True)

comparison = benchmark.compare(category, query)
print(comparison)

Fetched 1 rows in 0.007s
{'rows_difference': 0, 'time_difference': 0.005832580998685444, 'rows_improvement_percent': 0.0, 'time_improvement_percent': 45.98975678094913}


4 - Calcul RFM (Recency, Frequency, Monetary) pour chaque client

Points à optimiser:
- date(o.order_purchase_timestamp) empêche l’utilisation directe d’un index sur order_purchase_timestamp.
- Le CTE order_totals peut être remplacé par une sous-requête pour éviter que SQLite matérialise le CTE (utile sur de gros volumes).

Améliorations:
- SUBSTR(...,1,10) au lieu de date() → plus rapide et indexable.
- Suppression de CTE : SQLite optimisera mieux la sous-requête.
- COUNT(DISTINCT o.order_id) → sécurité si une commande a plusieurs lignes dans order_items.

In [48]:
category = "customers"
query = "rfmbbq_analysis"

execute('opti_pki', category, query, optimized=True)

comparison = benchmark.compare(category, query)
print(comparison)

Fetched 98666 rows in 0.619s
{'rows_difference': 0, 'time_difference': -0.10129961900383933, 'rows_improvement_percent': 0.0, 'time_improvement_percent': -19.578309459954703}


### 🚀 📊 Cohortes

1 - Analyse de cohortes pour suivre la rétention client mois après mois

Points à optimiser:
- date(o.order_purchase_timestamp) et strftime(...) empêchent l’utilisation d’index.
- CTE multiples → SQLite va souvent matérialiser chaque CTE, ce qui peut coûter cher.
- Les calculs de cohort_month peuvent être simplifiés en pré-calculant le nombre de mois.

Changements clés:
- SUBSTR(...,1,10) remplace date() → plus rapide, indexable si tu ajoutes un index sur order_purchase_timestamp.
- CAST(SUBSTR(...)) remplace strftime() pour calculer les mois → moins coûteux pour SQLite.
- Même logique de cohort_month, mais plus efficace pour gros volumes.

In [49]:
category = "cohorts"
query = "first_month_retention"

execute('opti_pki', category, query, optimized=True)

comparison = benchmark.compare(category, query)
print(comparison)

Fetched 634 rows in 0.129s
{'rows_difference': 0, 'time_difference': 0.2931561489967862, 'rows_improvement_percent': 0.0, 'time_improvement_percent': 69.4361015756003}


2 - LTV (Lifetime Value) par cohorte

A améliorer:
- Il y a plusieurs JOIN et date() sur chaque ligne.
- On groupe par o.order_id, o.customer_id, cohort_date → beaucoup de lignes intermédiaires.
- Les fonctions date() empêchent l’usage d’index.

Améliorations:
- SUBSTR(...,1,10) remplace date() → plus rapide et indexable.
- Même logique de groupements, mais simplifiée pour éviter des fonctions SQLite lourdes.
- COUNT(DISTINCT customer_id) pour éviter double comptage si un client a plusieurs commandes dans la cohorte.

In [50]:
category = "cohorts"
query = "lifetime_value_per_cohort"

execute('opti_pki', category, query, optimized=True)

comparison = benchmark.compare(category, query)
print(comparison)



Fetched 616 rows in 0.572s
{'rows_difference': 0, 'time_difference': 0.18657415999769, 'rows_improvement_percent': 0.0, 'time_improvement_percent': 24.595460461799828}
